# TODO: HashStack

In [1]:
from typing import Dict
import sys

import IPython.display
import pandas

sys.path.append('..')

import src.constants
import src.db
import src.swap_liquidity

## Load and prepare events

In [2]:
# Establish the connection.
connection = src.db.establish_connection()

# Load all HashStack events.
hashstack_events = pandas.read_sql(
    sql = 
    f"""
    SELECT
        *
    FROM
        starkscan_events
    WHERE
        from_address='{src.constants.Protocol.HASHSTACK.value}'
    AND
        key_name IN ('new_loan', 'loan_withdrawal', 'loan_repaid', 'loan_swap', 'collateral_added', 'collateral_withdrawal', 'loan_interest_deducted', 'liquidated')
    ORDER BY
        block_number, id ASC;
    """,
    con = connection,
)

# Close the connection.
connection.close()

/tmp/ipykernel_1185515/1239245774.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hashstack_events = pandas.read_sql(


In [3]:
hashstack_events.set_index('id', inplace = True)

In [4]:
# TODO: ensure we're processing loan_repaid after all other loan-altering events + other events in "logical" order
hashstack_events['order'] = hashstack_events['key_name'].map(
    {
        'new_loan': 0,
        'loan_withdrawal': 3,
        'loan_repaid': 4,
        'loan_swap': 1,
        'collateral_added': 6,
        'collateral_withdrawal': 7,
        'loan_interest_deducted': 5,
        'liquidated': 2,
    },
)
hashstack_events.sort_values(['block_number', 'transaction_hash', 'order'], inplace = True)

In [5]:
# TODO
print({x: len(hashstack_events[hashstack_events['key_name'] == x]) for x in hashstack_events['key_name'].unique()})

{'new_loan': 130, 'loan_withdrawal': 48, 'loan_interest_deducted': 177, 'loan_swap': 178, 'collateral_added': 32, 'loan_repaid': 93, 'collateral_withdrawal': 92, 'liquidated': 2}


# Process events

TODO

Events examples:
- 'OwnershipTransferred': not needed?
- 'ModuleFunctionChange': not needed?
- 'dial_parameters': not needed?
- 'savings_apr_update': not needed?
- 'borrow_apr_update': not needed?
- 'RoleGranted': not needed?
- 'RoleAdminChanged': not needed?
- 'RoleRevoked': not needed?

- 'liquidationState': not needed?
- 'smart_liquidation': not needed?

- [new_deposit](https://starkscan.co/event/0x07f3dbd47f1f73f88e4738952b926ca8f1ce3d0e3ab7b06274563aa87c96861d_2): not needed?
- [deposit_added](https://starkscan.co/event/0x008298bf990ac4988b889264883bfb082b5a8e3638e557a89e07a919f9faa8a7_3): not needed?
- [deposit_apr_accrued](https://starkscan.co/event/0x0471d0d9056c3623e4f76ff4f3a26d808c3f7d6b7f36d098eb3ceca9be89641b_0): not needed?
- [deposit_withdrawal](https://starkscan.co/event/0x02a2541942d3fe4f7af4b578f87aa13507beff31db954847dd3b8e9600904903_3): not needed?

- [new_loan](https://starkscan.co/event/0x04ff9acb9154603f1fc14df328a3ea53a6c58087aaac0bfbe9cc7f2565777db8_2)
- [loan_withdrawal](https://starkscan.co/event/0x05bb8614095fac1ac9b405c27e7ce870804e85aa5924ef2494fec46792b6b8dc_2)
- [loan_repaid](https://starkscan.co/event/0x07731e48d33f6b916f4e4e81e9cee1d282e20e970717e11ad440f73cc1a73484_1)
- [loan_swap](https://starkscan.co/event/0x00ad0b6b00ce68a1d7f5b79cd550d7f4a15b1708b632b88985a4f6faeb42d5b1_7)

- [collateral_added](https://starkscan.co/event/0x02df71b02fce15f2770533328d1e645b957ac347d96bd730466a2e087f24ee07_2)
- [collateral_withdrawal](https://starkscan.co/event/0x03809ebcaad1647f2c6d5294706e0dc619317c240b5554848c454683a18b75ba_5)
- [loan_interest_deducted](https://starkscan.co/event/0x050db0ed93d7abbfb152e16608d4cf4dbe0b686b134f890dd0ad8418b203c580_2)

- [liquidated](https://starkscan.co/event/0x0774bebd15505d3f950c362d813dc81c6320ae92cb396b6469fd1ac5d8ff62dc_8)

In [6]:
# TODO
import collections
import decimal


class HashStackBorrowings:
    """
    TODO
    """

    def __init__(
        self,
        borrowings_id: int,
        market: str,
        amount: decimal.Decimal,
        current_market: str,
        current_amount: decimal.Decimal,
        debt_category: int,
    ) -> None:
        self.id: int = borrowings_id
#         self.owner  # TODO: needed?
        self.market: str = market
#         self.commitment  # TODO: needed?
        self.amount: decimal.Decimal = amount
        self.current_market: str = current_market
        self.current_amount: decimal.Decimal = current_amount
#         self.is_loan_withdrawn  # TODO needed?
        self.debt_category: int = debt_category
#         self.state  # TODO: needed?
#         self.l3_integration  # TODO: needed?
#         self.created_at  # TODO: needed?


class HashStackCollateral:
    """
    TODO
    """

    def __init__(
        self,
        market: str,
        amount: decimal.Decimal,
        current_amount: decimal.Decimal,
    ) -> None:
        self.market: str = market
        self.amount: decimal.Decimal = amount
        self.current_amount: decimal.Decimal = current_amount
#         self.commitment  # TODO: needed?
#         self.timelock_validity  # TODO: needed?
#         self.is_timelock_activated  # TODO: needed?
#         self.activation_time  # TODO: needed?


class HashStackLoan:
    """
    TODO
    """

    def __init__(
        self,
        borrowings: HashStackBorrowings,
        collateral: HashStackCollateral,
    ) -> None:
        # TODO: save user?
        self.borrowings: HashStackBorrowings = borrowings
        self.collateral: HashStackCollateral = collateral


class UserState:
    """
    TODO
    """

    def __init__(self) -> None:
        self.loans: Dict[int, HashStackLoan] = {}


class State:
    """
    TODO
    """

    # TODO: f'process_{name.lower()}_event'?
    EVENTS_FUNCTIONS_MAPPING: Dict[str, str] = {
        "new_loan": "process_new_loan_event",
        # TODO: this event shows what the user does with the loan, but it shouldn't change the amount borrowed, so let's ignore it for now
        "loan_withdrawal": "process_loan_withdrawal_event",
        "loan_repaid": "process_loan_repaid_event",
        "loan_swap": "process_loan_swap_event",
        "collateral_added": "process_collateral_added_event",
        "collateral_withdrawal": "process_collateral_withdrawal_event",
        "loan_interest_deducted": "process_loan_interest_deducted_event",
        "liquidated": "process_liquidated_event",
    }
#     USER = '0x3139000ef6bd54e3fb2e70149e2281b660c642fdf39158fe096c9e0f662dd32'  # TODO
#     USER = '0x7e436ddaaac0790ba140d1294922cc3ea8d98622c2488525cd52b0a461565d6'  # TODO: user with no collateral??
#     USER = '0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f507b4ca728d8e427b7864'  # TODO: first user .. has a loan but no collateral?
    USER = '0x7b027f9df477d0f96bf8dbc4774f8af954d9f00d0e44ffd29fea09f783ca453'
#     USER = '0x724102a5654d669ba7ce25e420e733ddb409b753598ab0883ad476cf9faed7b'

    def __init__(self) -> None:
        # TODO: how to compute the interest accrued on both collateral and loan?
        self.user_loan_ids_mapping: collections.defaultdict = collections.defaultdict(list)
        self.user_states: collections.defaultdict = collections.defaultdict(UserState)

    def process_event(self, event: pandas.Series) -> None:
        # TODO: filter events in the query
        if not event["key_name"] in self.EVENTS_FUNCTIONS_MAPPING:
            return
        getattr(self, self.EVENTS_FUNCTIONS_MAPPING[event["key_name"]])(event=event)

    def process_new_loan_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, 
        # `current_amount`, `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`,
        # `market`, `amount`, `current_amount`, `commitment`, `timelock_validity`, `is_timelock_activated`,
        # `activation_time`, `timestamp`.
        loan_id = int(event["data"][0], base = 16)
        user = event["data"][1]
        self.user_loan_ids_mapping[user].append(loan_id)
        # TODO: universal naming: use deposit, collateral and debt (instead of loan/borrowings)?
        borrowings_token = src.constants.get_symbol(event["data"][2])
        borrowings_amount = decimal.Decimal(str(int(event["data"][4], base=16)))
        borrowings_current_token = src.constants.get_symbol(event["data"][6])
        borrowings_current_amount = decimal.Decimal(str(int(event["data"][7], base=16)))
        debt_category = int(event["data"][10], base=16)
        # TODO: first ~3 loans seem to have different structure of 'data'
        try:
            collateral_token = src.constants.get_symbol(event["data"][14])
            collateral_amount = decimal.Decimal(str(int(event["data"][15], base=16)))
            collateral_current_amount = decimal.Decimal(str(int(event["data"][17], base=16)))
        except KeyError:
            collateral_token = src.constants.get_symbol(event["data"][13])
            collateral_amount = decimal.Decimal(str(int(event["data"][14], base=16)))
            collateral_current_amount = decimal.Decimal(str(int(event["data"][16], base=16)))
        self.user_states[user].loans[loan_id] = HashStackLoan(
            borrowings = HashStackBorrowings(
                borrowings_id = loan_id,
                market = borrowings_token,
                amount = borrowings_amount,
                current_market = borrowings_current_token,
                current_amount = borrowings_current_amount,
                debt_category = debt_category,
            ),
            collateral = HashStackCollateral(
                market = collateral_token,
                amount = collateral_amount,
                current_amount = collateral_current_amount,
            ),
        )
        # TODO
        if user == self.USER:
            print(
                event['block_number'],
                'new_loan \n borrowings',
                vars(self.user_states[user].loans[loan_id].borrowings),
                '\n collateral',
                vars(self.user_states[user].loans[loan_id].collateral),
            )

    def process_loan_withdrawal_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, 
        # `current_amount`, `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`,
        # `amount_withdrawn`, `timestamp`.
        loan_id = int(event["data"][0], base = 16)
        user = event["data"][1]
        token = src.constants.get_symbol(event["data"][2])
        amount = decimal.Decimal(str(int(event["data"][4], base=16)))
        current_token = src.constants.get_symbol(event["data"][6])
        current_amount = decimal.Decimal(str(int(event["data"][7], base=16)))
        debt_category = int(event["data"][10], base=16)
        self.user_states[user].loans[loan_id].borrowings = HashStackBorrowings(
            borrowings_id = loan_id,
            market = token,
            amount = amount,
            current_market = current_token,
            current_amount = current_amount,
            debt_category = debt_category,
        )
        # TODO
        if user == self.USER:
            print(event['block_number'], 'loan_withdrawal \n borrowings', vars(self.user_states[user].loans[loan_id].borrowings))

    def process_loan_repaid_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, 
        # `current_amount`, `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`,
        # `timestamp`.
        loan_id = int(event["data"][0], base = 16)
        user = event["data"][1]
        token = src.constants.get_symbol(event["data"][2])
        amount = decimal.Decimal(str(int(event["data"][4], base=16)))
        current_token = src.constants.get_symbol(event["data"][6])
        current_amount = decimal.Decimal(str(int(event["data"][7], base=16)))
        # TODO: from the docs it seems that it's only possible to repay the whole amount
        assert current_amount == decimal.Decimal('0')
        debt_category = int(event["data"][10], base=16)
        self.user_states[user].loans[loan_id].borrowings = HashStackBorrowings(
            borrowings_id = loan_id,
            market = token,
            amount = decimal.Decimal("0"),  # TODO: this should be set to 0
            current_market = current_token,
            current_amount = current_amount,
            debt_category = debt_category,
        )
        # TODO
        if user == self.USER:
            print(event['block_number'], 'loan_repaid \n borrowings', vars(self.user_states[user].loans[loan_id].borrowings))

    def process_loan_swap_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, 
        # `current_amount`, `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`,
        # `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, `current_amount`,
        # `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`, `timestamp`.
        old_loan_id = int(event["data"][0], base = 16)
        old_user = event["data"][1]
        assert old_loan_id in self.user_loan_ids_mapping[old_user]
        new_loan_id = int(event["data"][14], base = 16)
        new_user = event["data"][15]
        assert old_loan_id == new_loan_id
        # TODO: this doesn't always have to hold, right?
        assert old_user == new_user
        # TODO: universal naming: use deposit, collateral and debt (instead of loan/borrowings)?
#         old_token = src.constants.get_symbol(event["data"][2])  # TODO: not needed?
#         old_amount = decimal.Decimal(str(int(event["data"][4], base=16)))  # TODO: not needed?
#         old_current_token = src.constants.get_symbol(event["data"][6])  # TODO: not needed?
#         old_current_amount = decimal.Decimal(str(int(event["data"][7], base=16)))  # TODO: not needed?
        new_token = src.constants.get_symbol(event["data"][16])
        new_amount = decimal.Decimal(str(int(event["data"][18], base=16)))
        assert self.user_states[old_user].loans[old_loan_id].borrowings.market == new_token
        assert self.user_states[old_user].loans[old_loan_id].borrowings.amount == new_amount
        new_current_token = src.constants.get_symbol(event["data"][20])
        new_current_amount = decimal.Decimal(str(int(event["data"][21], base=16)))
        old_debt_category = int(event["data"][10], base=16)
        new_debt_category = int(event["data"][24], base=16)
        # TODO: this need to hold, right?
        assert old_debt_category == new_debt_category
        # TODO: from the docs it seems that it's only possible to swap the whole balance
        self.user_states[new_user].loans[new_loan_id].borrowings = HashStackBorrowings(
            borrowings_id = new_loan_id,
            market = new_token,
            amount = new_amount,
            current_market = new_current_token,
            current_amount = new_current_amount,
            debt_category = new_debt_category,
        )
        # TODO
        if new_user == self.USER:
            print(event['block_number'], 'loan_swap \n borrowings', vars(self.user_states[new_user].loans[new_loan_id].borrowings))

    def process_collateral_added_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `market`, `amount`, `current_amount`, `commitment`,
        # `timelock_validity`, `is_timelock_activated`, `activation_time`, `loan_id`, `amount_added`,
        # `timestamp`.
        loan_id = int(event["data"][9], base = 16)
        # TODO: create method self.find_user?
        users = [user for user, loan_ids in self.user_loan_ids_mapping.items() if loan_id in loan_ids]
        assert len(users) == 1
        user = users[0]
        # TODO: universal naming: use deposit, collateral and debt (instead of loan/borrowings)?
        token = src.constants.get_symbol(event["data"][0])
        amount = decimal.Decimal(str(int(event["data"][1], base=16)))  # TODO: needed?
        current_amount = decimal.Decimal(str(int(event["data"][3], base=16)))
        # TODO: utilize `amount_added`?
        self.user_states[user].loans[loan_id].collateral = HashStackCollateral(
            market = token,
            amount = amount,
            current_amount = current_amount,
        )
        # TODO
        if user == self.USER:
            print(event['block_number'], 'collateral_added \n collateral', vars(self.user_states[user].loans[loan_id].collateral))

    def process_collateral_withdrawal_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `market`, `amount`, `current_amount`, `commitment`,
        # `timelock_validity`, `is_timelock_activated`, `activation_time`, `loan_id`, `amount_withdrawn`,
        # `timestamp`.
        loan_id = int(event["data"][9], base = 16)
        # TODO: create method self.find_user?
        users = [user for user, loan_ids in self.user_loan_ids_mapping.items() if loan_id in loan_ids]
        assert len(users) == 1
        user = users[0]
        # TODO: universal naming: use deposit, collateral and debt (instead of loan/borrowings)?
        token = src.constants.get_symbol(event["data"][0])
        amount = decimal.Decimal(str(int(event["data"][1], base=16)))  # TODO: needed?
        current_amount = decimal.Decimal(str(int(event["data"][3], base=16)))
        # TODO: utilize `amount_withdrawn`?
        self.user_states[user].loans[loan_id].collateral = HashStackCollateral(
            market = token,
            amount = amount,
            current_amount = current_amount,
        )
        # TODO
        if user == self.USER:
            print(event['block_number'], 'collateral_withdrawal \n collateral', vars(self.user_states[user].loans[loan_id].collateral))

    def process_loan_interest_deducted_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `market`, `amount`, `current_amount`, `commitment`,
        # `timelock_validity`, `is_timelock_activated`, `activation_time`, `accrued_interest`, `loan_id`,
        # `timestamp`.
        loan_id = int(event["data"][11], base = 16)
        # TODO: create method self.find_user?
        users = [user for user, loan_ids in self.user_loan_ids_mapping.items() if loan_id in loan_ids]
        assert len(users) == 1
        user = users[0]
        # TODO: universal naming: use deposit, collateral and debt (instead of loan/borrowings)?
        token = src.constants.get_symbol(event["data"][0])
        amount = decimal.Decimal(str(int(event["data"][1], base=16)))  # TODO: needed?
        current_amount = decimal.Decimal(str(int(event["data"][3], base=16)))
        # TODO: utilize `amount_withdrawn`?
        self.user_states[user].loans[loan_id].collateral = HashStackCollateral(
            market = token,
            amount = amount,
            current_amount = current_amount,
        )
        # TODO
        if user == self.USER:
            print(event['block_number'], 'loan_interest_deducted \n collateral', vars(self.user_states[user].loans[loan_id].collateral))

    def process_liquidated_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, 
        # `current_amount`, `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`,
        # `liquidator`, `timestamp`.
        loan_id = int(event["data"][0], base = 16)
        user = event["data"][1]
        token = src.constants.get_symbol(event["data"][2])
        amount = decimal.Decimal(str(int(event["data"][4], base=16)))
        current_token = src.constants.get_symbol(event["data"][6])
        current_amount = decimal.Decimal(str(int(event["data"][7], base=16)))
        # TODO: from the docs it seems that it's only possible to liquidate the whole amount
        assert current_amount == decimal.Decimal('0')
        debt_category = int(event["data"][10], base=16)
        self.user_states[user].loans[loan_id].borrowings = HashStackBorrowings(
            borrowings_id = loan_id,
            market = token,
            amount = amount,
            current_market = current_token,
            current_amount = current_amount,
            debt_category = debt_category,
        )
        # TODO: what happens to the collateral? now assuming it disappears
        self.user_states[user].loans[loan_id].collateral = HashStackCollateral(
            market = self.user_states[user].loans[loan_id].collateral.market,
            amount = self.user_states[user].loans[loan_id].collateral.amount,
            current_amount = decimal.Decimal('0'),
        )
        # TODO
        if user == self.USER:
            print(
                event['block_number'],
                'liquidated \n borrowings',
                vars(self.user_states[user].loans[loan_id].borrowings),
                '\n collateral',
                vars(self.user_states[user].loans[loan_id].collateral),
            )

In [7]:
# Iterate over ordered events to obtain the final state of each user.
state = State()
# bns = set()  # TODO
for _, event in hashstack_events.iterrows():
#     bns.update({event['block_number']})  # TODO
    # TODO: save the timestamp/block number of each update?
    state.process_event(event = event)
# sorted([x for x in range(min(bns), max(bns)) if x + 1 not in bns])[-200:]  # TODO

38312 new_loan 
 borrowings {'id': 61, 'market': 'USDC', 'amount': Decimal('30000000'), 'current_market': 'USDC', 'current_amount': Decimal('29970000'), 'debt_category': 2} 
 collateral {'market': 'ETH', 'amount': Decimal('10000000000000000'), 'current_amount': Decimal('10000000000000000')}
38317 loan_repaid 
 borrowings {'id': 61, 'market': 'USDC', 'amount': Decimal('0'), 'current_market': 'USDC', 'current_amount': Decimal('0'), 'debt_category': 2}
38317 loan_interest_deducted 
 collateral {'market': 'ETH', 'amount': Decimal('10000000000000000'), 'current_amount': Decimal('9999939758408005')}
38317 collateral_withdrawal 
 collateral {'market': 'ETH', 'amount': Decimal('10000000000000000'), 'current_amount': Decimal('0')}


In [8]:
# TODO: just a simple sanity check
for user, user_state in state.user_states.items():
    for loan_id, loan in user_state.loans.items():
        if loan.collateral.current_amount < 0:
            print('negative collateral')
        if loan.borrowings.current_amount < 0:
            print('negative borrowings')
#         print(loan_id, 'c:', loan.collateral.market, loan.collateral.current_amount, 'b:', loan.borrowings.current_market, loan.borrowings.current_amount)

In [9]:
# Get current prices of tokens.
prices = src.swap_liquidity.Prices()

# Compute health factors

TODO: This part is new. Will be refactored.

In [10]:
# TODO
from typing import Dict
import collections
import decimal


# Source: Starkscan, e.g. 
# https://starkscan.co/token/0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7 for ETH.
TOKEN_DECIMAL_FACTORS = {
    "ETH": decimal.Decimal('1e18'),
    "wBTC": decimal.Decimal('1e8'),
    "USDC": decimal.Decimal('1e6'),
    "DAI": decimal.Decimal('1e18'),
    "USDT": decimal.Decimal('1e6'),
}


def compute_collateral_current_amount_usd(
    collateral: HashStackCollateral,
    prices: Dict[str, decimal.Decimal],
) -> decimal.Decimal:
    return (
        collateral.current_amount
        * prices[collateral.market]
        # TODO: perform the conversion using TOKEN_DECIMAL_FACTORS sooner (in `UserTokenState`?)?
        / TOKEN_DECIMAL_FACTORS[collateral.market]
    )


def compute_borrowings_current_amount_usd(
    borrowings: HashStackBorrowings,
    prices: Dict[str, decimal.Decimal],
) -> decimal.Decimal:
    return (
        borrowings.current_amount
        * prices[borrowings.current_market]
        # TODO: perform the conversion using TOKEN_DECIMAL_FACTORS sooner (in `UserTokenState`?)?
        / TOKEN_DECIMAL_FACTORS[borrowings.current_market]
    )


def compute_borrowings_amount_usd(
    borrowings: HashStackBorrowings,
    prices: Dict[str, decimal.Decimal],
) -> decimal.Decimal:
    return (
        borrowings.amount
        * prices[borrowings.market]
        # TODO: perform the conversion using TOKEN_DECIMAL_FACTORS sooner (in `UserTokenState`?)?
        / TOKEN_DECIMAL_FACTORS[borrowings.market]
    )


def compute_health_factor(
    collateral: HashStackCollateral,
    borrowings: HashStackBorrowings,
    prices: Dict[str, decimal.Decimal],
    user: str,
) -> decimal.Decimal:
    collateral_current_amount_usd = compute_collateral_current_amount_usd(collateral = collateral, prices = prices)
    borrowings_current_amount_usd = compute_borrowings_current_amount_usd(borrowings = borrowings, prices = prices)
    borrowings_amount_usd = compute_borrowings_amount_usd(borrowings = borrowings, prices = prices)

    # TODO: how can this happen?
    if borrowings_amount_usd == decimal.Decimal('0'):
        # TODO: assumes collateral is positive
        return decimal.Decimal('Inf')

    health_factor = (collateral_current_amount_usd + borrowings_current_amount_usd) / borrowings_amount_usd
    # TODO: enable?
#     if health_factor < decimal.Decimal('0.9'):
#         print(f'Suspiciously low health factor = {health_factor} of user = {user}, investigate.', collateral_current_amount_usd, borrowings_current_amount_usd, borrowings_amount_usd)
    # TODO: too many loans eligible for liquidation?
    # TODO: this should be a method of the borrowings class
    health_factor_liquidation_threshold = decimal.Decimal('1.06') if loan.borrowings.debt_category == 1 \
        else decimal.Decimal('1.05') if loan.borrowings.debt_category == 2 \
        else decimal.Decimal('1.04')
    # TODO: enable?
#     if health_factor >= health_factor_liquidation_threshold:
#         # TODO: loan with hf = ... of user = ... eligible ...
#         print(f'Health factor = {health_factor} of user = {user} eligible for liquidation.')
    return health_factor

In [11]:
# TODO
loan_stats = pandas.DataFrame()
# TODO: use [(user, loan_id) for user, user_state in state.user_states.items() for loan_id in user_state.loans.keys()]?
loan_stats['User'] = [user for user, user_state in state.user_states.items() for _ in user_state.loans.keys()]
loan_stats['Loan ID'] = [loan_id for user_state in state.user_states.values() for loan_id in user_state.loans.keys()]
loan_stats['Borrowings: amount in USD'] = loan_stats.apply(
    lambda x: compute_borrowings_amount_usd(borrowings = state.user_states[x['User']].loans[x['Loan ID']].borrowings, prices = prices.prices),
    axis = 1,
)
loan_stats['Borrowings: current amount in USD'] = loan_stats.apply(
    lambda x: compute_borrowings_current_amount_usd(borrowings = state.user_states[x['User']].loans[x['Loan ID']].borrowings, prices = prices.prices),
    axis = 1,
)
loan_stats['Collateral: current amount in USD'] = loan_stats.apply(
    lambda x: compute_collateral_current_amount_usd(collateral = state.user_states[x['User']].loans[x['Loan ID']].collateral, prices = prices.prices),
    axis = 1,
)
loan_stats['Health factor'] = loan_stats.apply(
    lambda x: compute_health_factor(
        borrowings = state.user_states[x['User']].loans[x['Loan ID']].borrowings,
        collateral = state.user_states[x['User']].loans[x['Loan ID']].collateral,
        prices = prices.prices,
        user = x['User'],
    ),
    axis = 1,
)
loan_stats

,User,Loan ID,Borrowings: amount in USD,Borrowings: current amount in USD,Collateral: current amount in USD,Health factor
0,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f50...,1,0E-59,0E-59,0E-59,Infinity
1,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f50...,2,28.93392000000000098225427791,29.59618161645299892831173505,15.000000,1.541311430198638740079186959
2,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f50...,3,30.000000,29.970000,16.07440000000000054569682106,1.534813333333333351523227369
3,0x1e307316999c4830ada67b495d03246ab205e77853bb...,4,0E-59,0E-59,0E-59,Infinity
4,0x1e307316999c4830ada67b495d03246ab205e77853bb...,18,0E-59,0E-59,0E-59,Infinity
...,...,...,...,...,...,...
125,0x62513b05f2d77687ecdca05805afc5b580bb951cc2fe...,123,30.15328655490299890813876704,30.12313423075199890923059343,10.05109551830099963604625568,1.332333365250381620844490629
126,0x3bc6af96c742680064e2fd89635cf60e7b3b38269ea5...,124,273.367234,273.093867,80.37200000000000272848410532,1.293007438484745405619768261
127,0x3942c0f2ff317206f7757c224605d5b1b617090bfdb4...,126,30.000000,29.510176,11.647198,1.371912466666666666666666667
128,0x1dc0f785c9182c00392ccd4759c55fa1ff37daebd8b1...,128,30.478851,30.448373,10.159617,1.332333361254333373656375695


In [24]:
hashstack_loan_stats = pandas.DataFrame()
hashstack_loan_stats['User'] = [user for user, user_state in state.user_states.items() for _ in user_state.loans.keys()]
hashstack_loan_stats['Loan ID'] = [loan_id for user_state in state.user_states.values() for loan_id in user_state.loans.keys()]
hashstack_loan_stats['Borrowing in USD'] = hashstack_loan_stats.apply(
    lambda x: compute_borrowings_amount_usd(borrowings = state.user_states[x['User']].loans[x['Loan ID']].borrowings, prices = prices.prices),
    axis = 1,
)
hashstack_loan_stats = hashstack_loan_stats[hashstack_loan_stats['Borrowing in USD'] > decimal.Decimal("0")]
hashstack_loan_stats['Risk adjusted collateral in USD'] = hashstack_loan_stats.apply(
    lambda x: 
        compute_collateral_current_amount_usd(collateral = state.user_states[x['User']].loans[x['Loan ID']].collateral, prices = prices.prices)
        + compute_borrowings_current_amount_usd(borrowings = state.user_states[x['User']].loans[x['Loan ID']].borrowings, prices = prices.prices),
    axis = 1,
)
hashstack_loan_stats['Health factor'] = hashstack_loan_stats.apply(
    lambda x: compute_health_factor(
        borrowings = state.user_states[x['User']].loans[x['Loan ID']].borrowings,
        collateral = state.user_states[x['User']].loans[x['Loan ID']].collateral,
        prices = prices.prices,
        user = x['User'],
    ),
    axis = 1,
)


def get_collateral_str(loan: HashStackLoan) -> str:
    if loan.collateral.market == loan.borrowings.current_market:
        return (
            str(loan.collateral.market)
            + ": "
            + str(
                format(
                    (
                        loan.collateral.amount
                        + loan.borrowings.current_amount
                    ) / (
                        10 ** src.constants.symbol_decimals_map[loan.collateral.market]
                    ),
                    ".4f",
                )
            )
        )
    return (
        str(loan.collateral.market)
        + ": "
        + str(
            format(
                loan.collateral.amount / (
                    10 ** src.constants.symbol_decimals_map[loan.collateral.market]
                ),
                ".4f",
            )
        )
        + ", "
        + str(loan.borrowings.current_market)
        + ": "
        + str(
            format(
                loan.borrowings.current_amount / (
                    10 ** src.constants.symbol_decimals_map[loan.borrowings.current_market]
                ),
                ".4f",
            )
        )
    )


hashstack_loan_stats['Collateral'] = hashstack_loan_stats.apply(
    lambda x: get_collateral_str(loan = state.user_states[x['User']].loans[x['Loan ID']]),
    axis = 1,
)
hashstack_loan_stats['Borrowings'] = hashstack_loan_stats.apply(
    lambda x: (
        str(state.user_states[x['User']].loans[x['Loan ID']].borrowings.market)
        + ": "
        + str(
            format(
                state.user_states[x["User"]].loans[x["Loan ID"]].borrowings.amount / (
                    10 ** src.constants.symbol_decimals_map[state.user_states[x['User']].loans[x['Loan ID']].borrowings.market]
                ),
                ".4f",
            )
        )
    ),
    axis = 1,
)
hashstack_loan_stats.drop(columns = ["Loan ID"], inplace = True)
# hashstack_loan_stats.loc[
#     hashstack_loan_stats['Borrowing in USD'] >= decimal.Decimal('100')
# ].sort_values('Health factor').iloc[:20].to_csv("hashstack_data/large_loans_sample.csv", index=False, compression='gzip')
# hashstack_loan_stats.loc[
#     hashstack_loan_stats['Borrowing in USD'] < decimal.Decimal('100')
# ].sort_values('Health factor').iloc[:20].to_csv("hashstack_data/small_loans_sample.csv", index=False, compression='gzip')

hashstack_loan_stats

,User,Borrowing in USD,Risk adjusted collateral in USD,Health factor,Collateral,Borrowings
1,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f50...,28.93392000000000098225427791,44.59618161645299892831173505,1.541311430198638740079186959,"USDC: 15.0000, USDT: 29.6145",ETH: 0.0180
2,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f50...,30.000000,46.04440000000000054569682106,1.534813333333333351523227369,"ETH: 0.0100, USDC: 29.9700",USDC: 30.0000
7,0x74061d07a0fbd8ffc376dc4f593c69854c51b6c2fe55...,32.14880000000000109139364213,46.40550141931794008787684861,1.443459831138889741218769168,"ETH: 0.0100, USDT: 30.3499",ETH: 0.0200
11,0x1caf85dbb2e8c6d551f7d94d5acc9f854680ec060f7a...,32.14880000000000109139364213,40.15385119999981490011065901,1.248999999999994200000000000,ETH: 0.0250,ETH: 0.0200
13,0x1caf85dbb2e8c6d551f7d94d5acc9f854680ec060f7a...,29.98142999999999891436175403,37.30707902000000049112713896,1.244339546846164503895569424,"ETH: 0.0090, wBTC: 0.0009",USDT: 30.0000
23,0x3163a2390c34b10c3ac51f063af99204f3466357e884...,31.000000,41.96219099999999960193264315,1.353619064516129019417182037,"USDT: 11.0000, USDC: 30.9690",USDC: 31.0000
25,0x732f5f56f0a0a1888a9db1f35bc729595f6c62c492e0...,30.79962222531899888473472249,9.594794902738023408125941695,0.3115231359834851959837032261,"ETH: 0.0060, USDT: 0.0000",USDT: 30.8187
29,0x43e083db1638fd89a9f77f0d935a4bed785d4e638ef1...,35.56666959531899871211824138,37.46101336174159774495722385,1.053261769740507765888464322,"ETH: 0.0065, USDT: 27.0021",USDT: 35.5887
30,0x7fb5b4be6a89550d9539d807bcd54dbd4e4bd1bf924e...,28.93392000000000098225427791,63.85876888000000216788976104,2.207055555555555555555555556,ETH: 0.0397,ETH: 0.0180
45,0x724102a5654d669ba7ce25e420e733ddb409b753598a...,166.528973,48.07187802774390461923335579,0.2886697561495434468285188776,"ETH: 0.0300, USDC: 0.0000",USDC: 166.5290


In [27]:
# TODO
import plotly.express


figure = plotly.express.histogram(
    loan_stats.loc[loan_stats['Health factor'] < 5],#.astype(float),
    x = 'Health factor',
    nbins = 100,
    title = 'Health factor histogram',
)
figure.show()

In [16]:
# TODO: check numbers of borrowings-collateral token pairs
import itertools


counts = {}
for b, c in itertools.product(['wBTC', 'ETH', 'USDC', 'DAI', 'USDT'], ['wBTC', 'ETH', 'USDC', 'DAI', 'USDT']):
    count = 0
    for user, user_state in state.user_states.items():
        for loan_id, loan in user_state.loans.items():
            if loan.borrowings.market == b and loan.collateral.market == c:
                count += 1
    counts[f'{b} - {c}'] = count
counts

{'wBTC - wBTC': 1,
 'wBTC - ETH': 1,
 'wBTC - USDC': 0,
 'wBTC - DAI': 0,
 'wBTC - USDT': 1,
 'ETH - wBTC': 0,
 'ETH - ETH': 23,
 'ETH - USDC': 2,
 'ETH - DAI': 0,
 'ETH - USDT': 7,
 'USDC - wBTC': 1,
 'USDC - ETH': 21,
 'USDC - USDC': 19,
 'USDC - DAI': 1,
 'USDC - USDT': 3,
 'DAI - wBTC': 0,
 'DAI - ETH': 4,
 'DAI - USDC': 2,
 'DAI - DAI': 0,
 'DAI - USDT': 1,
 'USDT - wBTC': 0,
 'USDT - ETH': 26,
 'USDT - USDC': 8,
 'USDT - DAI': 0,
 'USDT - USDT': 9}

# Compute max liquidated borrowings

TODO: This part is new. Will be refactored.

In [17]:
# TODO
# TODO: compute_health_factor, etc. should be methods of class UserState
def compute_borrowings_to_be_liquidated(
    risk_adjusted_collateral_usd: decimal.Decimal,
    borrowings_usd: decimal.Decimal,
    borrowings_token_price: decimal.Decimal,
    collateral_token_collateral_factor: decimal.Decimal,
    collateral_token_liquidation_bonus: decimal.Decimal,
) -> decimal.Decimal:
    # TODO: commit the derivation of the formula in a document?
    numerator = borrowings_usd - risk_adjusted_collateral_usd
    denominator = borrowings_token_price * (
        1 - collateral_token_collateral_factor * (
            1 + collateral_token_liquidation_bonus
        )
    )
    return numerator / denominator


def compute_max_liquidated_amount(
    state: State,
    prices: Dict[str, decimal.Decimal],
    borrowings_token: str,
) -> decimal.Decimal:
    liquidated_borrowings_amount = decimal.Decimal('0')
    for user, user_state in state.user_states.items():
        for loan_id, loan in user_state.loans.items():
            # TODO: do this?
            # Filter out users who borrowed the token of interest.
            if borrowings_token != loan.borrowings.market:
                continue

            # Filter out users with health below 1.
            borrowings_amount_usd = compute_borrowings_amount_usd(borrowings = loan.borrowings, prices = prices)
            health_factor = compute_health_factor(borrowings = loan.borrowings, collateral = loan.collateral, prices = prices, user = user)
            # TODO: this should be a method of the borrowings class
            health_factor_liquidation_threshold = decimal.Decimal('1.06') if loan.borrowings.debt_category == 1 \
                else decimal.Decimal('1.05') if loan.borrowings.debt_category == 2 \
                else decimal.Decimal('1.04')
            if health_factor >= health_factor_liquidation_threshold:
                continue

            # TODO: find out how much of the borrowings_token will be liquidated
            liquidated_borrowings_amount += borrowings_amount_usd
    return liquidated_borrowings_amount

## Single price change

In [18]:
# TODO
# TODO: adjust prices and observe the amounts
import copy


COLLATERAL_TOKEN = 'ETH'
BORROWINGS_TOKEN = 'USDC'
COLLATERAL_TOKEN_PRICE = decimal.Decimal('1500')


def simulate_liquidations_under_absolute_price_change(
    prices: src.swap_liquidity.Prices,
    collateral_token: str,
    collateral_token_price: decimal.Decimal,
    state: State,
    borrowings_token: str,
) -> decimal.Decimal:
    changed_prices = copy.deepcopy(prices.prices)
    changed_prices[collateral_token] = collateral_token_price
    return compute_max_liquidated_amount(state = state, prices = changed_prices, borrowings_token = borrowings_token)


simulate_liquidations_under_absolute_price_change(
    prices = prices,
    collateral_token = COLLATERAL_TOKEN,
    collateral_token_price = COLLATERAL_TOKEN_PRICE,
    state = state,
    borrowings_token = BORROWINGS_TOKEN,
)

Decimal('196.695093')

## Range of price changes

### Nominal price change

In [19]:
# TODO: adjust prices and observe the amounts
import numpy


COLLATERAL_TOKEN = 'ETH'
BORROWINGS_TOKEN = 'USDT'


def decimal_range(start: decimal.Decimal, stop: decimal.Decimal, step: decimal.Decimal):
    while start < stop:
        yield start
        start += step


data = pandas.DataFrame(
    {
        'collateral_token_price': [
            x
            for x
            in decimal_range(
                # TODO: make it dependent on the collateral token .. use prices.prices[COLLATERAL_TOKEN]
                start = decimal.Decimal('0'),
                stop = decimal.Decimal('3000'),
                # TODO: make it dependent on the collateral token
                step = decimal.Decimal('50'),
            )
        ]
    },
)
data['max_borrowings_to_be_liquidated'] = \
    data['collateral_token_price'].apply(
        lambda x: simulate_liquidations_under_absolute_price_change(
            prices = prices,
            collateral_token = COLLATERAL_TOKEN,
            collateral_token_price = x,
            state = state,
            borrowings_token = BORROWINGS_TOKEN,
        )
    )

data['max_borrowings_to_be_liquidated_at_interval'] = data['max_borrowings_to_be_liquidated'].diff().abs()
# TODO: drops also other NaNs, if there are any
data.dropna(inplace = True)
# TODO: the numbers seem weird
data

,collateral_token_price,max_borrowings_to_be_liquidated,max_borrowings_to_be_liquidated_at_interval
1,50,126.3291518206379954255764719,0E-25
2,100,126.3291518206379954255764719,0E-25
3,150,126.3291518206379954255764719,0E-25
4,200,96.34772182063799651121471790,29.98142999999999891436175400
5,250,96.34772182063799651121471790,0E-26
6,300,96.34772182063799651121471790,0E-26
7,350,96.34772182063799651121471790,0E-26
8,400,96.34772182063799651121471790,0E-26
9,450,96.34772182063799651121471790,0E-26
10,500,96.34772182063799651121471790,0E-26


In [23]:
# Setup the AMM.
swap_amms = await src.swap_liquidity.SwapAmm().init()


# TODO: this function is general for all protocols
def get_amm_supply_at_price(
    collateral_token: str,
    collateral_token_price: decimal.Decimal,
    debt_token: str,
    amm: src.swap_liquidity.SwapAmm,
) -> decimal.Decimal:
    return amm.get_pool(collateral_token, debt_token).supply_at_price(
        debt_token, collateral_token_price
    )


data['amm_borrowings_token_supply'] = \
    data['collateral_token_price'].apply(
        lambda x: get_amm_supply_at_price(
            collateral_token = COLLATERAL_TOKEN,
            collateral_token_price = x,
            borrowings_token = BORROWINGS_TOKEN,
            amm=swap_amms,
        )
    )
data

TypeError: get_amm_supply_at_price() got an unexpected keyword argument 'borrowings_token'

## Plot the liquidated and supply amounts

In [ ]:
import plotly.express


figure = plotly.express.bar(
    data.astype(float),
    x = 'collateral_token_price',
    y = ['max_borrowings_to_be_liquidated_at_interval', 'amm_borrowings_token_supply'],
    title = f'Potentially liquidatable amounts of {BORROWINGS_TOKEN} and the corresponding supply',
    barmode = 'overlay',
    opacity = 0.65,
)
figure.show()